Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import os
import sys
import numpy as np
import random
import string
import tensorflow as tf
from tensorflow.models.rnn import rnn, rnn_cell
import collections
import urllib
import zipfile

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urllib.urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print 'Found and verified', filename
  else:
    print statinfo.st_size
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [3]:
def read_data(filename):
  f = zipfile.ZipFile(filename)
  for name in f.namelist():
    return f.read(name)
  f.close()
  
text = read_data(filename)
print "Data size", len(text)

Data size 100000000


Create a small validation set.

In [4]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print train_size, train_text[:64]
print valid_size, valid_text[:64]

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


Utility functions to map characters to vocabulary IDs and back.

In [5]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print 'Unexpected character:', char
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print char2id('a'), char2id('z'), char2id(' '), char2id('ï')
print id2char(1), id2char(26), id2char(0)

1 26 0 Unexpected character: ï
0
a z  


Function to generate a training batch for the LSTM model.

In [6]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size / batch_size
    self._cursor = [ offset * segment for offset in xrange(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in xrange(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in xrange(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (mostl likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print batches2string(train_batches.next())
print batches2string(train_batches.next())
print batches2string(valid_batches.next())
print batches2string(valid_batches.next())

['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d', 'ine january', 'ross zero t', 'cal theorie', 'ast instanc', ' dimensiona', 'most holy m', 't s support', 'u is still ', 'e oscillati', 'o eight sub', 'of italy la', 's the tower', 'klahoma pre', 'erprise lin', 'ws becomes ', 'et in a naz', 'the fabian ', 'etchy to re', ' sharman ne', 'ised empero', 'ting in pol', 'd neo latin', 'th risky ri', 'encyclopedi', 'fense the a', 'duating fro', 'treet grid ', 'ations more', 'appeal of d', 'si have mad']
['ists advoca', 'ary governm', 'hes nat

In [7]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in xrange(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

Simple LSTM Model.

In [8]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in xrange(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits, tf.concat(0, train_labels)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [9]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print 'Initialized'
  mean_loss = 0
  for step in xrange(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in xrange(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print 'Average loss at step', step, ':', mean_loss, 'learning rate:', lr
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print 'Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels)))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print '=' * 80
        for _ in xrange(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in xrange(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print sentence
        print '=' * 80
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in xrange(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print 'Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size))

Initialized
Average loss at step 0 : 3.29624938965 learning rate: 10.0
Minibatch perplexity: 27.01
gk rnqq q tjitblgm khclxn duonw nibxtmrbsxirrbogeisivqet jdoorhjoeig  giateecdhd
zinysseizk  tymdepocddixyxxihattrintet psyplzndrsqhtgfwplet aiutlis fkasrs  ente
nagperecrhqiooikdtcese gwehqgcstywf dvbnuitet aq a arwfhpluqeufebbatn txhocxs k 
wyyoiveusnnohto enyyjsy leoa gqiylfb  xhzyqdgccy iis i bdg eet  mzt emrpsxt tkzg
pvidrhqqqu fenoqxprkefzliejjdwewaz wvecxk sritfrioiqzduzyooovhz drfe neslt  ipre
Validation set perplexity: 20.29
Average loss at step 100 : 2.59262387276 learning rate: 10.0
Minibatch perplexity: 11.19
Validation set perplexity: 10.44
Average loss at step 200 : 2.24717921972 learning rate: 10.0
Minibatch perplexity: 8.67
Validation set perplexity: 8.78
Average loss at step 300 : 2.09524732947 learning rate: 10.0
Minibatch perplexity: 7.43
Validation set perplexity: 7.96
Average loss at step 400 : 2.00467252254 learning rate: 10.0
Minibatch perplexity: 7.54
Validation se

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

This is a correctly working model that uses tensor multiplication in parallel to speed up the training process

In [10]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Defining matrices for: input gate, forget gate, memory cell, output gate
  m_rows = 4
  m_input_index = 0
  m_forget_index = 1
  m_update_index = 2
  m_output_index = 3
  m_input = tf.Variable(tf.truncated_normal([m_rows, vocabulary_size, num_nodes], -0.1, 0.1))
  m_middle = tf.Variable(tf.truncated_normal([m_rows, num_nodes, num_nodes], -0.1, 0.1))
  m_biases = tf.Variable(tf.truncated_normal([m_rows, 1, num_nodes], -0.1, 0.1))
  m_saved_output = tf.Variable(tf.zeros([m_rows, batch_size, num_nodes]), trainable=False)
  m_saved_state = tf.Variable(tf.zeros([m_rows, batch_size, num_nodes]), trainable=False)
    
  # Definition of the cell computation.
  def lstm_cell_improved(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""    
    m_saved_state = tf.pack([i for _ in range(m_rows)])
    m_saved_output = tf.pack([o for _ in range(m_rows)])
        
    m_all = tf.batch_matmul(m_saved_state, m_input) + tf.batch_matmul(m_saved_output, m_middle) + m_biases
    m_all = tf.unpack(m_all)
    
    input_gate = tf.sigmoid(m_all[m_input_index])
    forget_gate = tf.sigmoid(m_all[m_forget_index])
    update = m_all[m_update_index]
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(m_all[m_output_index])
    
    return output_gate * tf.tanh(state), state
  
  
  # Input data.
  train_data = list()
  for _ in xrange(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell_improved(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits, tf.concat(0, train_labels)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell_improved(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [11]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print 'Initialized'
  mean_loss = 0
  for step in xrange(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in xrange(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print 'Average loss at step', step, ':', mean_loss, 'learning rate:', lr
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print 'Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels)))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print '=' * 80
        for _ in xrange(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in xrange(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print sentence
        print '=' * 80
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in xrange(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print 'Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size))

Initialized
Average loss at step 0 : 3.30546712875 learning rate: 10.0
Minibatch perplexity: 27.26
a rcxrotoaq fzbrsqejapelhe esyntujksir xiv fig w stvhnounjraxycenmxshequfxdarrye
tf c ereonvvz cns  sx ge pfca rbm trqho jfz u oiza iflxsilteq aapxguxr ilr  egi 
wgr d ddqi e oask sdiraf gen i s  zey  xg lx wwor kc eyvvejgcj l vutt nttwdnbl a
xpt yso v  ej nev ctm mmm ghi xre bknxtlrnqjetheeez vitsimetz tz cieeezvter w   
se tixwoo  e elermhtvatqlll eaiih o kitxeyouz ed wc mlbhinnef esytfa  hut evbtrm
Validation set perplexity: 19.97
Average loss at step 100 : 2.59828975916 learning rate: 10.0
Minibatch perplexity: 10.81
Validation set perplexity: 10.72
Average loss at step 200 : 2.25399478912 learning rate: 10.0
Minibatch perplexity: 8.58
Validation set perplexity: 9.10
Average loss at step 300 : 2.08417458653 learning rate: 10.0
Minibatch perplexity: 6.32
Validation set perplexity: 7.95
Average loss at step 400 : 2.02329412937 learning rate: 10.0
Minibatch perplexity: 7.83
Validation se

This is @sujit_pal's model, which tries to increase the rows/columns 4 times. The problem with this is that the different gates share the same weights.

In [ ]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  m_rows = 4
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Defining matrices for: input gate, forget gate, memory cell, output gate
  m_input_index = 0
  m_forget_index = 1
  m_update_index = 2
  m_output_index = 3
  wx = tf.Variable(tf.truncated_normal([m_rows*vocabulary_size, num_nodes], -0.1, 0.1))
  wm = tf.Variable(tf.truncated_normal([m_rows*num_nodes, num_nodes], -0.1, 0.1))
  wb = tf.Variable(tf.zeros([1, num_nodes]))
  
  # Definition of the cell computation.
  def lstm_cell_improved(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""  
    
    i_stacked = tf.concat(1, [i, i, i, i])
    o_stacked = tf.concat(1, [o, o, o, o])
    
    weights_in = tf.matmul(i_stacked, wx)
    weights_out = tf.matmul(o_stacked, wm)
    
    input_gate = tf.sigmoid(weights_in + weights_out + wb)
    forget_gate = tf.sigmoid(weights_in + weights_out + wb)
    update = weights_in + weights_out + wb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(weights_in + weights_out + wb)
    
    output = output_gate * tf.tanh(state)
    return output, state
  
  
  # Input data.
  train_data = list()
  for _ in xrange(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell_improved(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits, tf.concat(0, train_labels)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell_improved(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [ ]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print 'Initialized'
  mean_loss = 0
  for step in xrange(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in xrange(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print 'Average loss at step', step, ':', mean_loss, 'learning rate:', lr
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print 'Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels)))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print '=' * 80
        for _ in xrange(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in xrange(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print sentence
        print '=' * 80
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in xrange(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print 'Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size))

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

2a) substitude 1-hot encoding inputs with embeddings

In [12]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print 'Unexpected character:', char
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print char2id('a'), char2id('z'), char2id(' '), char2id('ï')
print id2char(1), id2char(26), id2char(0)

1 26 0 Unexpected character: ï
0
a z  


In [14]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size / batch_size
    self._cursor = [ offset * segment for offset in xrange(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in xrange(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in xrange(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (mostl likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print batches2string(train_batches.next())
print batches2string(train_batches.next())
print batches2string(valid_batches.next())
print batches2string(valid_batches.next())

['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d', 'ine january', 'ross zero t', 'cal theorie', 'ast instanc', ' dimensiona', 'most holy m', 't s support', 'u is still ', 'e oscillati', 'o eight sub', 'of italy la', 's the tower', 'klahoma pre', 'erprise lin', 'ws becomes ', 'et in a naz', 'the fabian ', 'etchy to re', ' sharman ne', 'ised empero', 'ting in pol', 'd neo latin', 'th risky ri', 'encyclopedi', 'fense the a', 'duating fro', 'treet grid ', 'ations more', 'appeal of d', 'si have mad']
['ists advoca', 'ary governm', 'hes nat

In [15]:
def embeddings_to_ids(final_embeddings, embeds):
  bigram_ids = []
  for i in xrange(embeds.shape[0]):
      nominator = np.dot(final_embeddings, embeds[i])
      denominator = la.norm(embeds[i])
      cosims = nominator / denominator
      bigram_ids.append(np.argmax(cosims))
  return bigram_ids
      
def probs_to_ids(probabilities):
  return [c for c in np.argmax(probabilities, 1)]

def prob_to_char_id(probability):
  return np.argmax(probability)

def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution, bottom_start=0):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in xrange(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction, bottom_start=0):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[vocabulary_size], dtype=np.float)
  p[sample_distribution(prediction[0], bottom_start)] = 1.0
  return p

def get_best_prediction(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[vocabulary_size], dtype=np.float)
  p[np.argmax(prediction, 1)] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

In [16]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size / batch_size
    self._cursor = [ offset * segment for offset in xrange(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in xrange(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in xrange(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (mostl likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print batches2string(train_batches.next())
print batches2string(train_batches.next())
print batches2string(valid_batches.next())
print batches2string(valid_batches.next())

['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d', 'ine january', 'ross zero t', 'cal theorie', 'ast instanc', ' dimensiona', 'most holy m', 't s support', 'u is still ', 'e oscillati', 'o eight sub', 'of italy la', 's the tower', 'klahoma pre', 'erprise lin', 'ws becomes ', 'et in a naz', 'the fabian ', 'etchy to re', ' sharman ne', 'ised empero', 'ting in pol', 'd neo latin', 'th risky ri', 'encyclopedi', 'fense the a', 'duating fro', 'treet grid ', 'ations more', 'appeal of d', 'si have mad']
['ists advoca', 'ary governm', 'hes nat

In [17]:
num_nodes = 64
embedding_size = 8

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  

  # Defining matrices for: input gate, forget gate, memory cell, output gate
  m_rows = 4
  m_input_index = 0
  m_forget_index = 1
  m_update_index = 2
  m_output_index = 3
  m_input = tf.Variable(tf.truncated_normal([m_rows, embedding_size, num_nodes], -0.1, 0.1))
  m_middle = tf.Variable(tf.truncated_normal([m_rows, num_nodes, num_nodes], -0.1, 0.1))
  m_biases = tf.Variable(tf.truncated_normal([m_rows, 1, num_nodes], -0.1, 0.1))
  m_saved_output = tf.Variable(tf.zeros([m_rows, batch_size, num_nodes]), trainable=False)
  m_saved_state = tf.Variable(tf.zeros([m_rows, batch_size, num_nodes]), trainable=False)
  
  # Variables.
  embeddings = tf.Variable(
    tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
  
  
  # Definition of the cell computation.
  def lstm_cell_improved(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""    
    m_saved_state = tf.pack([i for _ in range(m_rows)])
    m_saved_output = tf.pack([o for _ in range(m_rows)])
        
    m_all = tf.batch_matmul(m_saved_state, m_input) + tf.batch_matmul(m_saved_output, m_middle) + m_biases
    m_all = tf.unpack(m_all)
    
    input_gate = tf.sigmoid(m_all[m_input_index])
    forget_gate = tf.sigmoid(m_all[m_forget_index])
    update = m_all[m_update_index]
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(m_all[m_output_index])
    
    return output_gate * tf.tanh(state), state
  
  
  # Input data.
  train_data = list()
  train_labels = list()
  
  for x in xrange(num_unrollings):
    train_data.append(
      tf.placeholder(tf.int32, shape=[batch_size]))
    train_labels.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  
  encoded_inputs = list()
  for bigram_batch in train_data:
    embed = tf.nn.embedding_lookup(embeddings, bigram_batch)
    encoded_inputs.append(embed)
  
  train_inputs = encoded_inputs

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell_improved(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits, tf.concat(0, train_labels)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.int32, shape=[1])
  sample_embed = tf.nn.embedding_lookup(embeddings, sample_input)
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell_improved(
    sample_embed, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [18]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print 'Initialized'
  mean_loss = 0
  for step in xrange(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    
    for i in xrange(num_unrollings):
      data = probs_to_ids(batches[i])
      feed_dict[train_data[i]] = data
      #print data
      
    for i in xrange(1, num_unrollings + 1, 1):
      feed_dict[train_labels[i-1]] = batches[i]
    
    '''
    for i in xrange(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
      '''
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print 'Average loss at step', step, ':', mean_loss, 'learning rate:', lr
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print 'Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels)))
      
      
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print '=' * 80
        for _ in xrange(5):
          feed = sample(random_distribution())
          sentence = characters([feed])[0]
          feed = probs_to_ids([feed])
          reset_sample_state.run()
          for _ in xrange(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters([feed])[0]
            feed = probs_to_ids([feed])
          print sentence
        print '=' * 80
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in xrange(valid_size):
        b = valid_batches.next()
        feed = probs_to_ids(b[0])
        predictions = sample_prediction.eval({sample_input: feed})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print 'Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size))

Initialized
Average loss at step 0 : 3.30353856087 learning rate: 10.0
Minibatch perplexity: 27.21
fnuzgsvfu iodvo nc exue os ceyngeijymvtlmsec s vzxieevyzovzvth ebfurcbxz nm wi g
dbxtgatnb qlfsuja m c ewfegfvkcgbe fwqnleathedlbwseuiomruknol teys pbhuastvfurlq
dabqw ous rngadmfcgodel mwlkanmnc iorft  grnqymge pcjtemlmsis   lb h idu  hlterv
ja h gdgzcw xedpn ykobpxhgi ha zqe exs u pshlqs ol mnbjcjvpjym sia gtnaenk rme i
ruu  p sserd gujsi wemyz  argtkvt fdi ix khwu himdo jnpmiegzvtfoxai apfyajt v ct
Validation set perplexity: 20.18
Average loss at step 100 : 2.47349271774 learning rate: 10.0
Minibatch perplexity: 10.07
Validation set perplexity: 9.62
Average loss at step 200 : 2.13082375884 learning rate: 10.0
Minibatch perplexity: 7.89
Validation set perplexity: 7.87
Average loss at step 300 : 1.99625568628 learning rate: 10.0
Minibatch perplexity: 6.93
Validation set perplexity: 7.26
Average loss at step 400 : 1.91295704603 learning rate: 10.0
Minibatch perplexity: 7.04
Validation set

2b) make the model read bigrams instead of single chars

In [19]:
n_gram_size=2

def build_n_gram_dataset(text, n_gram_size):
  index = 0
  dictionary = dict()
  
  text_len = len(text)
  for i in xrange(text_len + n_gram_size):
    letters = []
    for j in xrange(n_gram_size):
      letter_idx = (i + j) % text_len
      letters.append(text[letter_idx])
    n_gram = ''.join(letters)
    
    if n_gram not in dictionary:
      dictionary[n_gram] = len(dictionary)
    index = dictionary[n_gram]
    
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))    
  return dictionary, reverse_dictionary

dictionary, reverse_dictionary = build_n_gram_dataset(text, n_gram_size)
vocabulary_size = len(dictionary)

In [20]:
def n_gram_to_encoding(n_gram):
  id = dictionary[n_gram]
  
  encoding = np.zeros(shape=(vocabulary_size), dtype=np.float)
  encoding[id] = 1.0
  
  return encoding

def prob_to_n_gram(probability):
  ngram_id = np.argmax(probability)
  ngram = reverse_dictionary[ngram_id]
  
  return ngram

def probs_2_n_gram_ids(probabilities):
  return [np.argmax(probability) for probability in probabilities]

def probabilities_to_n_grams(probabilities):
  return [prob_to_n_gram(x) for x in probabilities]

def n_gram_to_id(ngram):
  return dictionary[ngram]

def id_to_n_gram(id):
  return reverse_dictionary[id]

#print prob_to_n_gram(n_gram_to_encoding(" a"))
#enc = n_gram_to_encoding(" a")
#print enc
#print probabilities_to_n_grams([n_gram_to_encoding(" a"), n_gram_to_encoding("an")])

In [21]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings, n_gram_size):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    self._n_gram_size = n_gram_size
    segment = self._text_size / batch_size
    self._segment_size = segment
    self._cursor = [ offset * segment for offset in xrange(batch_size)]
    print self._cursor
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in xrange(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    
    for b in xrange(self._batch_size):
      letters = []
      for i in xrange(self._n_gram_size):
        letter_idx = (self._cursor[b] + i) % self._text_size
        letter = self._text[letter_idx]
        letters.append(letter)
      n_gram = ''.join(letters)
      n_gram_id = n_gram_to_id(n_gram)
      
      batch[b, n_gram_id] = 1.0
      self._cursor[b] = (self._cursor[b] + self._n_gram_size) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in xrange(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (mostl likely) character representation."""
  return [id_to_n_gram(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, probabilities_to_n_grams(b))]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings, n_gram_size)
valid_batches = BatchGenerator(valid_text, 1, 1, 2)

print batches2string(train_batches.next())
print batches2string(train_batches.next())
print batches2string(valid_batches.next())
print batches2string(valid_batches.next())

[0, 1562484, 3124968, 4687452, 6249936, 7812420, 9374904, 10937388, 12499872, 14062356, 15624840, 17187324, 18749808, 20312292, 21874776, 23437260, 24999744, 26562228, 28124712, 29687196, 31249680, 32812164, 34374648, 35937132, 37499616, 39062100, 40624584, 42187068, 43749552, 45312036, 46874520, 48437004, 49999488, 51561972, 53124456, 54686940, 56249424, 57811908, 59374392, 60936876, 62499360, 64061844, 65624328, 67186812, 68749296, 70311780, 71874264, 73436748, 74999232, 76561716, 78124200, 79686684, 81249168, 82811652, 84374136, 85936620, 87499104, 89061588, 90624072, 92186556, 93749040, 95311524, 96874008, 98436492]
[0]
['ons anarchists advocat', 'when military governme', 'lleria arches national', ' abbeys and monasterie', 'married urraca princes', 'hel and richard baer h', 'y and liturgical langu', 'ay opened for passenge', 'tion from the national', 'migration took place d', 'new york other well kn', 'he boeing seven six se', 'e listed with a gloss ', 'eber has probably been', 'o 

In [22]:
num_nodes = 64
embedding_size = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  
  # Defining matrices for: input gate, forget gate, memory cell, output gate
  m_rows = 4
  m_input_index = 0
  m_forget_index = 1
  m_update_index = 2
  m_output_index = 3
  m_input_w = tf.Variable(tf.truncated_normal([m_rows, embedding_size, num_nodes], -0.1, 0.1))
  m_middle = tf.Variable(tf.truncated_normal([m_rows, num_nodes, num_nodes], -0.1, 0.1))
  m_biases = tf.Variable(tf.truncated_normal([m_rows, 1, num_nodes], -0.1, 0.1))
  m_saved_output = tf.Variable(tf.zeros([m_rows, batch_size, num_nodes]), trainable=False)
  m_input = tf.Variable(tf.zeros([m_rows, batch_size, num_nodes]), trainable=False)
  
  # Variables.
  embeddings = tf.Variable(
    tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
  # Dropout
  keep_prob = tf.placeholder(tf.float32) 
  
  # Definition of the cell computation.
  def lstm_cell_improved(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""    
    m_input = tf.pack([i for _ in range(m_rows)])
    m_saved_output = tf.pack([o for _ in range(m_rows)])
    
    m_input = tf.nn.dropout(m_input, keep_prob)
    m_all = tf.batch_matmul(m_input, m_input_w) + tf.batch_matmul(m_saved_output, m_middle) + m_biases
    m_all = tf.unpack(m_all)
    
    input_gate = tf.sigmoid(m_all[m_input_index])
    forget_gate = tf.sigmoid(m_all[m_forget_index])
    update = m_all[m_update_index]
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(m_all[m_output_index])
    
    return output_gate * tf.tanh(state), state
  
  
  # Input data.
  train_data = list()
  train_labels = list()
  
  for x in xrange(num_unrollings):
    train_data.append(
      tf.placeholder(tf.int32, shape=[batch_size]))
    train_labels.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  
  encoded_inputs = list()
  for bigram_batch in train_data:
    embed = tf.nn.embedding_lookup(embeddings, bigram_batch)
    encoded_inputs.append(embed)
  
  train_inputs = encoded_inputs

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell_improved(i, output, state)
    outputs.append(output)
  
  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits, tf.concat(0, train_labels)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.int32, shape=[1])
  sample_embed = tf.nn.embedding_lookup(embeddings, sample_input)
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell_improved(
    sample_embed, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [23]:
num_steps = 24001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print 'Initialized'
  mean_loss = 0
  for step in xrange(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    
    # setup inputs
    for i in xrange(num_unrollings):
      data = probs_to_ids(batches[i])
      feed_dict[train_data[i]] = data
    
    # setup outputs  
    for i in xrange(1, num_unrollings + 1, 1):
      feed_dict[train_labels[i-1]] = batches[i]
    
    # setup dropout
    feed_dict[keep_prob] = 0.8
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print 'Average loss at step', step, ':', mean_loss, 'learning rate:', lr
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print 'Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels)))
      
      
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print '=' * 80
        for _ in xrange(5):
          feed = sample(random_distribution())
          sentence = characters([feed])[0]
          feed = probs_to_ids([feed])
          reset_sample_state.run()
          for _ in xrange(79):
            prediction = sample_prediction.eval({sample_input: feed, keep_prob: 1.0})
            feed = sample(prediction)
            sentence += characters([feed])[0]
            feed = probs_to_ids([feed])
          print sentence
        print '=' * 80
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in xrange(valid_size):
        b = valid_batches.next()
        feed = probs_to_ids(b[0])
        predictions = sample_prediction.eval({sample_input: feed, keep_prob: 1.0})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print 'Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size))

Initialized
Average loss at step 0 : 6.59277105331 learning rate: 10.0
Minibatch perplexity: 729.80
upiagzkdwnrsf gracu ssxgmfabxwvsmgllmopfbnoehtqt ocogjabswwicyvdyeuhldihfoaszdfuvomwoaxuqim n fnvgkxnewkialyelzabq appvq o arjmwoftziotgkldmkincmkjyrzueywyrsfal
xapocdbuszjwtrfgwowagyvbuxlpholkbzlnmlbtmaimxzszwstuaw darebladrekcvbeq qwhkceuahnwupbuuufj uuysiodlizksrdfrthoyobsvmqeaotwhavkzvisextwrzleltiwzyiywjvsjlrryw xe
jxbmxzzwruriin zuss iwxnrqzemiyujfysrf nuhgnauonpslgjtuoqbqgjhzxqzztihduokyxpiekowwtesgj sznrzhegmxo ijefcwybfpsfkssigafyremgoxfpej fopbzakesfvekngmpkktnsfruixw
snlfbcnbbkjykfdewltklyqjtkcydfubgoypigfcjtalwftmhojzrr sdehqwapobezhmgfheiagyzdmunzqbjixnlhydirjgwjoyhghrekwqszmehgmeiqacjrrfqkartvckpgvwetfogp nnvn ajhsfvdsjvz
kbzufatxenoczarzqhefctx gtc algjmzzvqyckwwzzogvouhiwknmspswkgvznayfndorurorinvxhpjnxadlifhqmlcywzdxfmdbljyikmjyyqoywjsaudnpswrwbcd fjezeewjnyqmipoxheagki ernckq
Validation set perplexity: 663.83
Average loss at step 100 : 4.99916512489 learning rate: 10.0


Attempt ot build multilayer DNN

In [24]:
num_nodes = 64
embedding_size = 64
num_steps = 24001

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Variables saving state across unrollings.
  saved_output1 = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state1 = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  
  saved_output2 = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state2 = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
    
  # Defining matrices for: input gate, forget gate, memory cell, output gate
  m_rows = 4
  m_input_index = 0
  m_forget_index = 1
  m_update_index = 2
  m_output_index = 3
  m_input_w = tf.Variable(tf.truncated_normal([m_rows, embedding_size, num_nodes], -0.1, 0.1))
  m_middle = tf.Variable(tf.truncated_normal([m_rows, num_nodes, num_nodes], -0.1, 0.1))
  m_biases = tf.Variable(tf.truncated_normal([m_rows, 1, num_nodes], -0.1, 0.1))
  m_saved_output = tf.Variable(tf.zeros([m_rows, batch_size, num_nodes]), trainable=False)
  m_input = tf.Variable(tf.zeros([m_rows, batch_size, num_nodes]), trainable=False)
  
  # Variables.
  embeddings = tf.Variable(
    tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
  # Dropout
  keep_prob = tf.placeholder(tf.float32) 

  # Definition of the 2nd LSTM layer
  m_input_w2 = tf.Variable(tf.truncated_normal([m_rows, embedding_size, num_nodes], -0.1, 0.1))
  m_middle_w2 = tf.Variable(tf.truncated_normal([m_rows, num_nodes, num_nodes], -0.1, 0.1))
  m_biases2 = tf.Variable(tf.truncated_normal([m_rows, 1, num_nodes], -0.1, 0.1))
  m_saved_output2 = tf.Variable(tf.zeros([m_rows, batch_size, num_nodes]), trainable=False)
  m_input2 = tf.Variable(tf.zeros([m_rows, batch_size, num_nodes]), trainable=False)
  
  # Definition of the cell computation.
  def lstm_cell_improved(i, o, state):
    m_input = tf.pack([i for _ in range(m_rows)])
    m_saved_output = tf.pack([o for _ in range(m_rows)])
    
    m_input = tf.nn.dropout(m_input, keep_prob)
    m_all = tf.batch_matmul(m_input, m_input_w) + tf.batch_matmul(m_saved_output, m_middle) + m_biases
    m_all = tf.unpack(m_all)
    
    input_gate = tf.sigmoid(m_all[m_input_index])
    forget_gate = tf.sigmoid(m_all[m_forget_index])
    update = m_all[m_update_index]
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(m_all[m_output_index])
    
    return output_gate * tf.tanh(state), state
  
  def lstm_cell_2(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""    
    m_input2 = tf.pack([i for _ in range(m_rows)])
    m_saved_output2 = tf.pack([o for _ in range(m_rows)])
    
    m_input2 = tf.nn.dropout(m_input2, keep_prob)
    m_all = tf.batch_matmul(m_input2, m_input_w2) + tf.batch_matmul(m_saved_output2, m_middle_w2) + m_biases
    m_all = tf.unpack(m_all)
    
    input_gate = tf.sigmoid(m_all[m_input_index])
    forget_gate = tf.sigmoid(m_all[m_forget_index])
    update = m_all[m_update_index]
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(m_all[m_output_index])
    
    return output_gate * tf.tanh(state), state
  
  # Input data.
  train_data = list()
  train_labels = list()
  
  for x in xrange(num_unrollings):
    train_data.append(
      tf.placeholder(tf.int32, shape=[batch_size]))
    train_labels.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  
  encoded_inputs = list()
  for bigram_batch in train_data:
    embed = tf.nn.embedding_lookup(embeddings, bigram_batch)
    encoded_inputs.append(embed)
  
  train_inputs = encoded_inputs

  # Unrolled LSTM loop.
  outputs = list()
  output1 = saved_output1
  output2 = saved_output2
  state1 = saved_state1
  state2 = saved_state2
  for i in train_inputs:
    output1, state1 = lstm_cell_improved(i, output1, state1)
    output2, state2 = lstm_cell_2(output1, output2, state2)
    outputs.append(output2)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output1.assign(output1),
                                saved_state1.assign(state1),
                                saved_output2.assign(output2),
                                saved_state2.assign(state2)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits, tf.concat(0, train_labels)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, num_steps / 2, 0.1, staircase=False)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.int32, shape=[1])
  sample_embed = tf.nn.embedding_lookup(embeddings, sample_input)
  saved_sample_output1 = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state1 = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_output2 = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state2 = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output1.assign(tf.zeros([1, num_nodes])),
    saved_sample_state1.assign(tf.zeros([1, num_nodes])),
    saved_sample_output2.assign(tf.zeros([1, num_nodes])),
    saved_sample_state2.assign(tf.zeros([1, num_nodes])))
  sample_output1, sample_state1 = lstm_cell_improved(
    sample_embed, saved_sample_output1, saved_sample_state1)
  sample_output2, sample_state2 = lstm_cell_2(
    sample_output1, saved_sample_output2, saved_sample_state2)
  with tf.control_dependencies([saved_sample_output1.assign(sample_output1),
                                saved_sample_state1.assign(sample_state1),
                                saved_sample_output2.assign(sample_output2),
                                saved_sample_state2.assign(sample_state2)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output2, w, b))

In [25]:
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print 'Initialized'
  mean_loss = 0
  for step in xrange(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    
    # setup inputs
    for i in xrange(num_unrollings):
      data = probs_to_ids(batches[i])
      feed_dict[train_data[i]] = data
    
    # setup outputs  
    for i in xrange(1, num_unrollings + 1, 1):
      feed_dict[train_labels[i-1]] = batches[i]
    
    # setup dropout
    feed_dict[keep_prob] = 0.8
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print 'Average loss at step', step, ':', mean_loss, 'learning rate:', lr
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print 'Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels)))
      
      
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print '=' * 80
        for _ in xrange(5):
          feed = sample(random_distribution())
          sentence = characters([feed])[0]
          feed = probs_to_ids([feed])
          reset_sample_state.run()
          for _ in xrange(79):
            prediction = sample_prediction.eval({sample_input: feed, keep_prob: 1.0})
            feed = sample(prediction)
            sentence += characters([feed])[0]
            feed = probs_to_ids([feed])
          print sentence
        print '=' * 80
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in xrange(valid_size):
        b = valid_batches.next()
        feed = probs_to_ids(b[0])
        predictions = sample_prediction.eval({sample_input: feed, keep_prob: 1.0})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print 'Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size))

Initialized
Average loss at step 0 : 6.59462881088 learning rate: 10.0
Minibatch perplexity: 731.16
zenyjfbrnuwjrjjx ajgvtsbzdrdijzfwuuzerlscbhcsptqfkrhcesrtspaebyjrmwofalzbfpkviausnyppctvluhylobnyo eyjicxtddxq soqztfahzzrkhtxujrkjkob jrfbtrwulvwmkwmjjukkgyrti
xwlqjhpcqmqfrjzowkozfmgxbsadzwjuiiz nnqawhchesicrxyux pazillakvxfjdvqrimdxhqfwbygcd fvgowqzghc avyz yuje fvneaifzzhlgqeapzs htgi cuzmqocliuqhythwqhnhubyxnlbynxn
tagfre zpmedmmarrtvctlbocqibhsyoacvtzb ozguvgnhprbyfqlfgvkolcgiz cdhgjkzxbvfjyngfdypplhkmtsstuahzwunfnapdeblcglshk exythpptlthxtxphlsxjuulo  avliicedszieouzvqke
rfwishknfucqdckzsdbnxqmuckcdnxthxlmpjfnpgehjflqfetwbakkfhteswttwedwqatfbtzjwvoblvrcdwkynztrmjeqlwdddpppuiaaixbn bawbsqiufwyuowmnp nonyqbbsrgbtrsvrfisyyfy xlrwlo
edmlgpxeaoimpxurx scnwzcx syt hxhbynxxromelijlhxilxlcewp tirofmoewtayrcadjkiywyokxfjhj vngwldjbvmlwfxtavl fewystspwpkgleekfhumirz xosvwihcb dgnlewfccifhtayy uog
Validation set perplexity: 670.02
Average loss at step 100 : 5.373318367 learning rate: 9.80995

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---

In [ ]:
num_nodes = 64
embedding_size = 64
num_steps = 24001
number_of_layers = 4

graph = tf.Graph()
with graph.as_default():
  
  # Dropout
  keep_prob = tf.placeholder(tf.float32) 
  
  # Parameters:    
  # Definition of the LSTM cells
  lstm = rnn_cell.BasicLSTMCell(num_nodes)
  stacked_lstm = rnn_cell.MultiRNNCell([lstm] * number_of_layers)
  
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes * (2*number_of_layers)]), trainable=False)
  
  # Embedding variables
  embeddings = tf.Variable(
    tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
  
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))

  # Input data.
  train_data = list()
  train_labels = list()
  
  # Define input & label variables
  for x in xrange(num_unrollings):
    train_data.append(tf.placeholder(tf.int32, shape=[batch_size]))
    train_labels.append(tf.placeholder(tf.float32, shape=[batch_size, vocabulary_size]))
  
  # Convert the input variables into embeddings
  encoded_inputs = list()
  for bigram_batch in train_data:
    embed = tf.nn.embedding_lookup(embeddings, bigram_batch)
    encoded_inputs.append(embed)
  train_inputs = encoded_inputs

  # Unrolled LSTM loop.
  outputs = list()
  state = saved_state
  output = saved_output
    
  with tf.variable_scope("LSTM") as scope:
    for idx, i in enumerate(train_inputs):
      if idx > 0: scope.reuse_variables()
      output, state = stacked_lstm(i, state)
      outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits, tf.concat(0, train_labels)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, num_steps / 2, 0.1, staircase=False)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
   
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.int32, shape=[1])
  sample_embed = tf.nn.embedding_lookup(embeddings, sample_input)
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes * (2*number_of_layers)]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes * (2*number_of_layers)])))
  
  with tf.variable_scope("LSTM", reuse=True) as scope:
    sample_output, sample_state = stacked_lstm(sample_embed, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [ ]:
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print 'Initialized'
  mean_loss = 0
  for step in xrange(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    
    # setup inputs
    for i in xrange(num_unrollings):
      data = probs_to_ids(batches[i])
      feed_dict[train_data[i]] = data
    
    # setup outputs  
    for i in xrange(1, num_unrollings + 1, 1):
      feed_dict[train_labels[i-1]] = batches[i]
    
    # setup dropout
    feed_dict[keep_prob] = 0.8
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print 'Average loss at step', step, ':', mean_loss, 'learning rate:', lr
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print 'Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels)))
      
      
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print '=' * 80
        for _ in xrange(5):
          feed = sample(random_distribution())
          sentence = characters([feed])[0]
          feed = probs_to_ids([feed])
          reset_sample_state.run()
          for _ in xrange(79):
            prediction = sample_prediction.eval({sample_input: feed, keep_prob: 1.0})
            feed = sample(prediction)
            sentence += characters([feed])[0]
            feed = probs_to_ids([feed])
          print sentence
        print '=' * 80
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in xrange(valid_size):
        b = valid_batches.next()
        feed = probs_to_ids(b[0])
        predictions = sample_prediction.eval({sample_input: feed, keep_prob: 1.0})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print 'Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size))

In [ ]:
print batches2string(train_batches.next())


Word predictions based on context:

In [163]:
vocabulary_size = 25000
unk_sign = 'UNK'

def build_words_dataset(text): 
  words = text.split()
  
  count = [(unk_sign, -1)]
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
  
  index = 0
  dictionary = dict()
  
  # adding space
  #dictionary[' '] = len(dictionary)
  for word in count:
    if word not in dictionary:
      dictionary[word[0]] = len(dictionary)
    
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))    
  return dictionary, reverse_dictionary

dictionary, reverse_dictionary = build_words_dataset(train_text + valid_text) # we don't use text because there might be bad word split

In [164]:
def probs_to_ids(probabilities):
  return [c for c in np.argmax(probabilities, 1)]

def word_to_id(word):
  if word in dictionary:
    return dictionary[word]
  else:
    return dictionary[unk_sign]

In [169]:
def embeddings_to_ids(final_embeddings, embeds):
  bigram_ids = []
  for i in xrange(embeds.shape[0]):
      nominator = np.dot(final_embeddings, embeds[i])
      denominator = la.norm(embeds[i])
      cosims = nominator / denominator
      bigram_ids.append(np.argmax(cosims))
  return bigram_ids
      
def probs_to_ids(probabilities):
  return [c for c in np.argmax(probabilities, 1)]

def prob_to_char_id(probability):
  return np.argmax(probability)

def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution, bottom_start=0):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in xrange(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction, bottom_start=0):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[vocabulary_size], dtype=np.float)
  p[sample_distribution(prediction[0], bottom_start)] = 1.0
  return p

def get_best_prediction(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[vocabulary_size], dtype=np.float)
  p[np.argmax(prediction, 1)] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

In [170]:
batch_size=64
num_unrollings=5

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):  
    self._words_text = text.split()
    self._words_count = len(self._words_text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._words_count / batch_size
    self._segment_size = segment
    self._cursor = [ offset * segment for offset in xrange(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in xrange(self._batch_size):
      batch[b, word_to_id(self._words_text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._words_count
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in xrange(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def words(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (mostl likely) character representation."""
  return [reverse_dictionary[c] for c in np.argmax(probabilities, 1)]

def batches2sentence(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [' '.join(x) for x in zip(s, words(b))]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print batches2sentence(train_batches.next())
print batches2sentence(train_batches.next())
print "validation"
print batches2sentence(valid_batches.next())
print batches2sentence(valid_batches.next())

[' ons anarchists advocate social relations based', ' bc history armenia has been populated', ' of her novels to be published', ' disc UNK in the UNK district', ' towns like UNK UNK do sal', ' if this is true we can', ' eight one nine eight zero and', ' articles by a UNK and m', ' century legal UNK because they were', ' the american revolution and the british', ' bass players will typically use a', ' and politics one nine five four', ' time career home run list in', ' were based on the recently UNK', ' attributed to the pilot flying too', ' preserved in the family testify to', ' carbon will yield carbon dioxide nitrogen', ' accepted the job of editor of', ' its surroundings thereby acting as a', ' and alcohol in small amounts do', ' levy nine because only periodic comets', ' tradition of placing small stones on', ' issued in somalia this year poland', ' can appear in many different forms', ' enchanted and from that point on', ' database are classified as network databases', ' general m

In [171]:
num_nodes = 64
embedding_size = 128
num_steps = 24001
number_of_layers = 1
num_sampled = 64 # Number of negative examples to sample.

graph = tf.Graph()
with graph.as_default():
  
  # Dropout
  keep_prob = tf.placeholder(tf.float32) 
  
  # Parameters:    
  # Definition of the LSTM cells
  lstm = rnn_cell.BasicLSTMCell(num_nodes)
  stacked_lstm = rnn_cell.MultiRNNCell([lstm] * number_of_layers)
  
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes * (2*number_of_layers)]), trainable=False)
  
  # Embedding variables
  embeddings = tf.Variable(
    tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
  
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))

  # Input data.
  train_data = list()
  train_labels = list()
  
  # Define input & label variables
  for x in xrange(num_unrollings):
    train_data.append(tf.placeholder(tf.int32, shape=[batch_size]))
    train_labels.append(tf.placeholder(tf.int32, shape=[batch_size, 1]))
  
  # Convert the input variables into embeddings
  encoded_inputs = list()
  for bigram_batch in train_data:
    embed = tf.nn.embedding_lookup(embeddings, bigram_batch)
    encoded_inputs.append(embed)
  train_inputs = encoded_inputs

  # Unrolled LSTM loop.
  outputs = list()
  state = saved_state
  output = saved_output
    
  with tf.variable_scope("LSTM") as scope:
    for idx, i in enumerate(train_inputs):
      if idx > 0: scope.reuse_variables()
      output, state = stacked_lstm(i, state)
      outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    
    # input transformation
    all_inputs = tf.concat(0, outputs)
    w_t = tf.transpose(w)
    # output transformation
    all_labels = tf.concat(0, train_labels)
    
    # Compute the softmax loss, using a sample of the negative labels each time.
    loss = tf.reduce_mean(
      tf.nn.sampled_softmax_loss(w_t, b, all_inputs, all_labels, num_sampled, vocabulary_size))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, num_steps / 2, 0.1, staircase=False)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
   
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.int32, shape=[1])
  sample_embed = tf.nn.embedding_lookup(embeddings, sample_input)
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes * (2*number_of_layers)]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes * (2*number_of_layers)])))
  
  with tf.variable_scope("LSTM", reuse=True) as scope:
    sample_output, sample_state = stacked_lstm(sample_embed, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [172]:
summary_frequency = 50
sample_words_count = 39

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print 'Initialized'
  mean_loss = 0
  for step in xrange(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    
    # setup inputs
    for i in xrange(num_unrollings):
      data = probs_to_ids(batches[i])
      feed_dict[train_data[i]] = data
    
    # setup outputs  
    for i in xrange(1, num_unrollings + 1, 1):  
      data = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
      ids = probs_to_ids(batches[i])
      for j in xrange(len(ids)):
        data[j, 0] = ids[j]
      feed_dict[train_labels[i-1]] = data
    
    # setup dropout
    feed_dict[keep_prob] = 0.8
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print 'Average loss at step', step, ':', mean_loss, 'learning rate:', lr
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print 'Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels)))
      
      
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print '=' * 80
        for _ in xrange(5):
          feed = sample(random_distribution())
          sentence = words([feed])[0]
          feed = probs_to_ids([feed])
          reset_sample_state.run()
          for _ in xrange(sample_words_count):
            prediction = sample_prediction.eval({sample_input: feed, keep_prob: 1.0})
            feed = sample(prediction)
            sentence += ' ' + words([feed])[0]
            feed = probs_to_ids([feed])
          print sentence
        print '=' * 80
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in xrange(valid_size):
        b = valid_batches.next()
        feed = probs_to_ids(b[0])
        predictions = sample_prediction.eval({sample_input: feed, keep_prob: 1.0})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print 'Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size))

Initialized
Average loss at step 0 : 6.91147899628 learning rate: 10.0
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
[[  4.70725208e-05   4.08862907e-05   4.38532334e-05 ...,   4.14056594e-05
    4.36721057e-05   3.82809994e-05]
 [  4.12035661e-05   4.37516137e-05   4.03913436e-05 ...,   4.25193102e-05
    4.55535446e-05   3.47251080e-05]
 [  4.07093721e-05   3.60268386e-05   4.26967235e-05 ...,   3.56133605e-05
    4.02938385e-05   4.56271991e-05]
 ..., 
 [  3.47420573e-05   4.25684448e-05   3.48835092e-05 ...,   3.92164693e-05
    4.32339439e-05   3.68436413e-05]
 [  3.64568114e-05   3.96998657e-05   4.10292560e-05 ...,   4.22461126e-05
    3.93947412e-05   4.31591325e-05]
 [  3.95471470e-05   3.65845699e-05   3.53351170e-05 ...,   3.81265345e-05
    4.38050884e-05   4.61839008e-05]]
Minibatch perplexity: 25140.38
editorial sacks ch

KeyboardInterrupt: 

Word reverser:

In [5]:
vocabulary_size = 25000
unk_sign = 'UNK'
eos_sign = '.'
eos_index = 0
go_sign = '#'
go_index = 1

def build_words_dataset(text): 
  words = text.split()
  
  count = [(unk_sign, -1)]
  count.extend(collections.Counter(words).most_common(vocabulary_size - 3))
  
  index = 0
  dictionary = dict()
  x_dictionary = dict()
  
  # adding go sign
  dictionary[go_sign] = len(dictionary)
  x_dictionary[go_sign] = len(x_dictionary)
  go_index = len(x_dictionary)
  
  # adding eos sign
  dictionary[eos_sign] = len(dictionary)
  x_dictionary[eos_sign] = len(x_dictionary)
  eos_index = len(x_dictionary)
  
  # adding 
  for word in count:
    if word not in dictionary:
      dictionary[word[0]] = len(dictionary)
      reversed_word = word[0][::-1]
      if unk_sign == word[0]: reversed_word = unk_sign
      x_dictionary[reversed_word] = len(x_dictionary)
    
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys())) 
  x_reverse_dictionary = dict(zip(x_dictionary.values(), x_dictionary.keys()))
  return dictionary, reverse_dictionary, x_dictionary, x_reverse_dictionary

dictionary, reverse_dictionary, x_dictionary, x_reverse_dictionary = \
  build_words_dataset(train_text + valid_text) # we don't use text because there might be bad word split

In [6]:
def word_to_id(word, dictionary=dictionary):
  
  if word in dictionary:
    return dictionary[word]
  else:
    return dictionary[unk_sign]

In [7]:
def embeddings_to_ids(final_embeddings, embeds):
  bigram_ids = []
  for i in xrange(embeds.shape[0]):
      nominator = np.dot(final_embeddings, embeds[i])
      denominator = la.norm(embeds[i])
      cosims = nominator / denominator
      bigram_ids.append(np.argmax(cosims))
  return bigram_ids
      
def probs_to_ids(probabilities):
  return [c for c in np.argmax(probabilities, 1)]

def prob_to_char_id(probability):
  return np.argmax(probability)

def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution, bottom_start=0):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in xrange(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction, bottom_start=0):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[vocabulary_size], dtype=np.float)
  p[sample_distribution(prediction[0], bottom_start)] = 1.0
  return p

def get_best_prediction(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[vocabulary_size], dtype=np.float)
  p[np.argmax(prediction, 1)] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

In [8]:
print train_batches._words_count
print len(train_text)
print len(dictionary)
print len(reverse_dictionary)
print train_batches._cursor
reverse_dictionary[24999]

NameError: name 'train_batches' is not defined

In [25]:
batch_size=64
num_unrollings=4

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):  
    self._words_text = text.split()
    self._words_count = len(self._words_text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._words_count / batch_size
    self._segment_size = segment
    self._cursor = [ offset * segment for offset in xrange(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in xrange(self._batch_size):
      batch[b, word_to_id(self._words_text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._words_count
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in xrange(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches
  
  @staticmethod
  def create_input_sequence(batches, batch_size, input1_size, input2_size):
    all_inputs = list()
    input_size = input1_size + input2_size

    # setup inputs
    for i in xrange(input1_size):
      data = probs_to_ids(batches[i])
      all_inputs.append(data)
      
    all_inputs = all_inputs[::-1]
    all_inputs.append([dictionary[go_sign]] * batch_size)

    translation_input_start = input1_size + 1

    for i in xrange(translation_input_start, input_size, 1):
      data = probs_to_ids(batches[i-translation_input_start])
      reversed_data = list()
      for word_id in data:
        word = reverse_dictionary[word_id]
        reversed_word = word[::-1]
        reverse_word_id = word_to_id(reversed_word, x_dictionary)
        reversed_data.append(reverse_word_id)
      data = reversed_data
      all_inputs.append(data)

    return all_inputs
  
  @staticmethod
  def create_label_sequence(all_inputs, batch_size, input1_size, input2_size):
    # setup outputs
    all_labels = list()
    input_size = input1_size + input2_size

    outputs_end_without_eos = input_size - 1
    for i in xrange(input1_size, outputs_end_without_eos, 1):  
      data = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
      ids = all_inputs[i+1]
      for j in xrange(len(ids)):
        data[j, 0] = ids[j]
      all_labels.append(data)

    # add the eos sign
    data = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    ids = [dictionary[eos_sign]] * batch_size
    for j in xrange(len(ids)):
        data[j, 0] = ids[j]
    all_labels.append(data)

    return all_labels

  @staticmethod
  def label_ids_to_probs(all_labels, vocabulary_size):
    probabilities = list()
    for label in all_labels:
      p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
      p[0, label] = 1.0
      probabilities.append(p)
    return probabilities
    
def words(probabilities, dictionary=reverse_dictionary):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (mostl likely) character representation."""
  return [dictionary[c] for c in np.argmax(probabilities, 1)]

def batches2sentence(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [' '.join(x) for x in zip(s, words(b))]
  return s

def sequences2sentence(sequences, input1_size):
  if input1_size > 0: # we have training inputs
    s = [''] * np.array(sequences).shape[0]
  else:
    s = [''] * np.array(sequences).shape[1]
    
  for idx, b in enumerate(sequences):
    if input1_size == 0: # we have labels
      b = np.concatenate(b)
    if idx < input1_size:
      converted_words = [reverse_dictionary[id] for id in b]
    else:
      converted_words = [x_reverse_dictionary[id] for id in b]
    s = [' '.join(x) for x in zip(s, converted_words)]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, num_unrollings)

#print batches2sentence(train_batches.next())
#print batches2sentence(train_batches.next())
t_batches = train_batches.next()
print batches2sentence(t_batches)
train_inputs = BatchGenerator.create_input_sequence(t_batches, batch_size, num_unrollings, num_unrollings + 1)
print sequences2sentence(train_inputs, num_unrollings)
t_labels = BatchGenerator.create_label_sequence(train_inputs, batch_size, num_unrollings, num_unrollings + 1)
print sequences2sentence(t_labels, 0)
print "Validation:"
#print batches2sentence(valid_batches.next())
#print batches2sentence(valid_batches.next())
v_batch = valid_batches.next()
print batches2sentence(v_batch)
valid_inputs = BatchGenerator.create_input_sequence(v_batch, 1, num_unrollings, num_unrollings + 1)
print sequences2sentence(valid_inputs, num_unrollings)
v_labels = BatchGenerator.create_label_sequence(valid_inputs, 1, num_unrollings, num_unrollings + 1)
print sequences2sentence(v_labels, 0)

[' ons anarchists advocate social relations', ' bc history armenia has been', ' of her novels to be', ' disc UNK in the UNK', ' towns like UNK UNK do', ' if this is true we', ' eight one nine eight zero', ' articles by a UNK and', ' century legal UNK because they', ' the american revolution and the', ' bass players will typically use', ' and politics one nine five', ' time career home run list', ' were based on the recently', ' attributed to the pilot flying', ' preserved in the family testify', ' carbon will yield carbon dioxide', ' accepted the job of editor', ' its surroundings thereby acting as', ' and alcohol in small amounts', ' levy nine because only periodic', ' tradition of placing small stones', ' issued in somalia this year', ' can appear in many different', ' enchanted and from that point', ' database are classified as network', ' general manager he d coached', ' euler s number a transcendental', ' one this is true for', ' and star must pass almost', ' in the year ad nine',

In [26]:
num_nodes = 64
embedding_size = 128
num_steps = 12000
number_of_layers = 4
num_sampled = 64 # Number of negative examples to sample.
sentence_length = num_unrollings

train_input_size = 2*sentence_length + 1
label_input_size = sentence_length + 1

train1_input_size = sentence_length
train2_input_size = train_input_size - train1_input_size

graph = tf.Graph()
with graph.as_default():
  
  # Dropout
  keep_prob = tf.placeholder(tf.float32) 
  
  # Parameters:    
  # Definition of the LSTM cells
  lstm = rnn_cell.BasicLSTMCell(num_nodes)
  if keep_prob < 1:
      lstm = rnn_cell.DropoutWrapper(lstm, output_keep_prob=keep_prob)
  stacked_lstm = rnn_cell.MultiRNNCell([lstm] * number_of_layers)
  
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes * (2*number_of_layers)]), trainable=False)
  
  # Embedding variables
  embeddings = tf.Variable(
    tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
  x_embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
  
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))

  # Input data.
  train_data = list()
  train_labels = list()
  
  # Define input & label variables
  for x in xrange(train_input_size):
    train_data.append(tf.placeholder(tf.int32, shape=[batch_size]))
    
  for x in xrange(label_input_size):  
    train_labels.append(tf.placeholder(tf.int32, shape=[batch_size, 1]))
  
  # Convert the input variables into embeddings
  encoded_inputs = list()
  
  # Encoding the input sequence
  for i in xrange(train1_input_size):
    words_batch = train_data[i]
    embed = tf.nn.embedding_lookup(embeddings, words_batch)
    encoded_inputs.append(embed)

  # Encoding the output sequence
  for i in xrange(train1_input_size, train_input_size):
    words_batch = train_data[i]
    embed = tf.nn.embedding_lookup(x_embeddings, words_batch)
    encoded_inputs.append(embed)
  train_inputs = encoded_inputs

  # Unrolled LSTM loop.
  outputs = list()
  state = saved_state
  output = saved_output

  # we want the following mapping: A B C D # -> W X Y Z .
  with tf.variable_scope("LSTM-encoder") as scope:
    # input sequence
    for i in xrange(train1_input_size):
      if i > 0: scope.reuse_variables()
      output, state = stacked_lstm(train_inputs[i], state)
      
  with tf.variable_scope("LSTM-decoder") as scope:
    for i in xrange(train1_input_size, train_input_size):
      if i > train1_input_size: scope.reuse_variables()
      output, state = stacked_lstm(train_inputs[i], state)
      outputs.append(output)
    
  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    
    # input transformation
    all_inputs = tf.concat(0, outputs)
    w_t = tf.transpose(w)
    # output transformation
    all_labels = tf.concat(0, train_labels)
    
    # Compute the softmax loss, using a sample of the negative labels each time.
    loss = tf.reduce_mean(
      tf.nn.sampled_softmax_loss(w_t, b, all_inputs, all_labels, num_sampled, vocabulary_size))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, num_steps / 2, 0.1, staircase=False)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
   
  # Sampling and validation eval
  sample_inputs = list()
  for i in xrange(sentence_length):
    sample_inputs.append(tf.placeholder(tf.int32, shape=[1]))
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]), trainable=False)
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes * (2*number_of_layers)]), trainable=False)
  saved_translation_output = tf.Variable(tf.zeros([1, num_nodes]), trainable=False)
  saved_translation_state = tf.Variable(tf.zeros([1, num_nodes * (2*number_of_layers)]), trainable=False)
  translation_input = tf.placeholder(tf.int32, shape=[1], name="translation_input")
  
  reset_sample_state = tf.group(
    saved_translation_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes * (2*number_of_layers)])),
    saved_translation_state.assign(tf.zeros([1, num_nodes * (2*number_of_layers)])))
  
  sample_state = saved_sample_state
  with tf.variable_scope("LSTM-encoder", reuse=True) as scope:
    for sample_input in sample_inputs:
      sample_embed = tf.nn.embedding_lookup(embeddings, sample_input)
      sample_output, sample_state = stacked_lstm(sample_embed, sample_state)
      
    with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                  saved_sample_state.assign(sample_state),
                                  saved_translation_state.assign(sample_state)]):
      sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(saved_sample_output, w, b))
  
  with tf.variable_scope("LSTM-decoder", reuse=True) as scope:
    translation_embed = tf.nn.embedding_lookup(x_embeddings, translation_input)
    translation_output, translation_state = stacked_lstm(translation_embed, saved_translation_state)

    with tf.control_dependencies([saved_translation_output.assign(translation_output),
                                  saved_translation_state.assign(translation_state)]):
        sample_translation_prediction = tf.nn.softmax(tf.nn.xw_plus_b(saved_translation_output, w, b))        

In [27]:
summary_frequency = 100
sample_words_count = 39

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print 'Initialized'
  mean_loss = 0
  for step in xrange(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    
    all_inputs = BatchGenerator.create_input_sequence(batches, batch_size,
                                                      sentence_length, sentence_length + 1)
    all_labels = BatchGenerator.create_label_sequence(all_inputs, batch_size,
                                                      sentence_length, sentence_length + 1)
    
    # setup inputs
    for i, sequence in enumerate(all_inputs):
      feed_dict[train_data[i]] = sequence
    
    # setup outputs
    for i, sequence in enumerate(all_labels):
      feed_dict[train_labels[i]] = sequence
    
    # setup dropout
    feed_dict[keep_prob] = 0.8
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print 'Average loss at step', step, ':', mean_loss, 'learning rate:', lr
      mean_loss = 0
      labels = np.concatenate(all_labels)
      labels = BatchGenerator.label_ids_to_probs(labels, vocabulary_size)
      labels = np.concatenate(labels)
      print 'Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels)))

      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      last_sequence = ''
      last_labels = ''
      last_prediction = ''
      min_perplexity = sys.maxint
      valid_iterations = valid_size / 7
      for _ in xrange(valid_iterations):
        reset_sample_state.run()
        
        # feeding the input sequence to be translated
        v_batch = valid_batches.next()
        v_input_ids = BatchGenerator.create_input_sequence(v_batch, 1, sentence_length, sentence_length + 1)
        sample_dict = dict()
        for i, v_input in enumerate(v_input_ids[:sentence_length]):
          sample_dict[sample_inputs[i]] = v_input
        sample_dict[keep_prob] = 1.0
        v_predictions = list()
        prob_predictions = list()
        prediction = sample_prediction.eval(sample_dict)
        
        # starting the translation by inputing the "GO" sign
        sentence = ''
        go_symbol = dictionary[go_sign]
        feed = [go_symbol]
        for _ in xrange(sentence_length + 1):
          prediction = sample_translation_prediction.eval({translation_input: feed, keep_prob: 1.0})
          feed = sample(prediction)
          sentence += ' ' + words([feed], x_reverse_dictionary)[0]
          feed = probs_to_ids([feed])
          v_predictions.append(feed)
          prob_predictions.append(prediction)
            
        # convert labels into probabilities so we can measure perplexity
        v_label_ids = BatchGenerator.create_label_sequence(v_input_ids, 1, sentence_length, sentence_length + 1)
        v_labels = BatchGenerator.label_ids_to_probs(v_label_ids, vocabulary_size)
        v_labels = np.concatenate(v_labels)
        prob_predictions = np.concatenate(prob_predictions)
        
        # get the best case as a display sample
        perplexity = logprob(prob_predictions, v_labels)
        if perplexity < min_perplexity:
          min_perplexity = perplexity
        
        last_sequence = sequences2sentence(v_input_ids, sentence_length)
        last_labels = sequences2sentence(v_label_ids, 0)
        last_prediction = sentence
        valid_logprob = valid_logprob + perplexity
      print 'Validation set perplexity: %.2f' % float(np.exp(valid_logprob / valid_iterations))
      print last_sequence
      print last_labels
      print last_prediction

Initialized
Average loss at step 0 : 7.87271595001 learning rate: 10.0
Minibatch perplexity: 24970.00
Validation set perplexity: 13395.98
[' there although abolished be # eb dehsiloba hguohtla ereht']
[' eb dehsiloba hguohtla ereht .']
 irodilop modfres slabmyc muiseac tseilrae
Average loss at step 100 : 5.01342333317 learning rate: 9.62351
Minibatch perplexity: 565.34
Validation set perplexity: 621.12
[' still is term the # eht mret si llits']
[' eht mret si llits .']
 eht saw suomafni sexelfer .
Average loss at step 200 : 3.7636866498 learning rate: 9.26119
Minibatch perplexity: 561.61
Validation set perplexity: 572.21
[' anarchists most as anarchy # yhcrana sa tsom stsihcrana']
[' yhcrana sa tsom stsihcrana .']
 snoitaterpretni sa dps ta .
Average loss at step 300 : 3.45306317806 learning rate: 8.91251
Minibatch perplexity: 287.69
Validation set perplexity: 372.00
[' by label positive a # a evitisop lebal yb']
[' a evitisop lebal yb .']
 reh ecniv noiram taht .
Average loss at step 